In [1]:
sys.stderr = open(snakemake.log[0], "w")
import pysam
# import glob

# snakemake_input = glob.glob("../../results/benchmarking/assembly/*")
# snakemake_output = '../../conda'

#with open(snakemake_output, "w") as out:
with open(snakemake.output[0], "w") as out:
   print("Accession", "Contigs", 'Total contigs', 'Contig length', 'Reference length', 'Contig frac', 'Edit dist.', 'Edit frac', sep="\t")
   print("Accession", "Contigs", 'Total contigs', 'Contig length', 'Reference length', 'Contig frac', 'Edit dist.', 'Edit frac', sep="\t", file=out)
   sum_of_edit_dist = 0
   largest_no_contig = 0
   small_larg_cover = 1.0

   #for bam_file in snakemake_input:
   for bam_file in snakemake.input:
      current_contig = 1

      with pysam.AlignmentFile(bam_file, "rb") as samfile:
         total_contigs = samfile.count()
         accession = samfile.get_reference_name(0)
         
         largest_no_contig = total_contigs if total_contigs > largest_no_contig else largest_no_contig

      with pysam.AlignmentFile(bam_file, "rb") as samfile:
         ref_lengths = samfile.lengths[0]
         largest_coverage = 0
         for read in samfile.fetch():
            query_alignment_length = read.query_alignment_length
            frac = round(query_alignment_length / ref_lengths, 2)
            edit = read.get_tag("NM")
            sum_of_edit_dist = sum_of_edit_dist + int(edit)
            edit_frac = round(read.get_tag("NM")/query_alignment_length, 5)
            largest_coverage = frac if frac > largest_coverage else largest_coverage

            print(accession, current_contig, total_contigs, query_alignment_length, ref_lengths, frac, edit, edit_frac, sep="\t")
            print(accession, current_contig, total_contigs, query_alignment_length, ref_lengths, frac, edit, edit_frac, sep="\t", file=out)

            current_contig += 1
         small_larg_cover = largest_coverage if small_larg_cover>largest_coverage else small_larg_cover
   
   print('Largest number of contigs')
   print(largest_no_contig)
   print('Smallest largest coverage')
   print(small_larg_cover)
   print('Sum of edit distances')
   print(sum_of_edit_dist)
   print('Largest number of contigs', file=out)
   print(largest_no_contig, file=out)
   print('Smallest largest coverage', file=out)
   print(small_larg_cover, file=out)
   print('Sum of edit distances', file=out)
   print(sum_of_edit_dist, file=out)


Accession	Contigs	Total contigs	Contig length	Reference length	Contig frac	Edit dist.	Edit frac
MT345844.1	1	1	29776	29776	1.0	2	7e-05
MT873106.1	1	28	350	29854	0.01	0	0.0
MT873106.1	2	28	611	29854	0.02	0	0.0
MT873106.1	3	28	464	29854	0.02	0	0.0
MT873106.1	4	28	421	29854	0.01	0	0.0
MT873106.1	5	28	441	29854	0.01	0	0.0
MT873106.1	6	28	315	29854	0.01	0	0.0
MT873106.1	7	28	608	29854	0.02	0	0.0
MT873106.1	8	28	653	29854	0.02	0	0.0
MT873106.1	9	28	328	29854	0.01	0	0.0
MT873106.1	10	28	933	29854	0.03	0	0.0
MT873106.1	11	28	304	29854	0.01	0	0.0
MT873106.1	12	28	771	29854	0.03	0	0.0
MT873106.1	13	28	414	29854	0.01	0	0.0
MT873106.1	14	28	1772	29854	0.06	0	0.0
MT873106.1	15	28	371	29854	0.01	1	0.0027
MT873106.1	16	28	2336	29854	0.08	0	0.0
MT873106.1	17	28	1537	29854	0.05	1	0.00065
MT873106.1	18	28	546	29854	0.02	0	0.0
MT873106.1	19	28	1184	29854	0.04	0	0.0
MT873106.1	20	28	1024	29854	0.03	0	0.0
MT873106.1	21	28	879	29854	0.03	0	0.0
MT873106.1	22	28	1238	29854	0.04	0	0.0
MT873106.1	23	28	361	2985